Q: В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации? Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы.

A: Поскольку коллаборативная фильтрация основана на матричной факторизации, она учитывает только реальные user-item взаимодействия (купил/не купил), игнорируя остальные доступные параметры (бренды товаров, время совершения покупки, частота покупок, возраст пользователей, их социальный статус, etc). В связи с этим возникает проблема "холодного старта": выдачи рекомендаций новым или мало активным пользователям, а также рекомендация новых товаров. И в предыдущих работах этого курса можно заметить, что некоторые алгоритмы выдают меньше предсказаний, чем необходимо. Гибридная рекомендательная система решает эти проблемы.

Q: Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/ Нам интересна именно рекомендательная система, раздел “Производительность системы” можно пропустить Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?

A: Описанная рекомендательная система HH состоит (в общем виде) из 4-х элементов (см. рис. Схема работы рекомендательной системы). На вебинарах мы рассматривали состоящие только из 2-х (ALS similar-user + item-item). К сожалению, в статье нет подробного описания работы каждого этапа моделирования, но в качестве примера сложностей в продакшене предположу следующее: эвристический фильтр на 2-х признаках и фльтрующая модель 1 - на 4-х - работают, скорее всего быстро, но насколько качественно - вопрос. Также не очень понятно, как решается проблема индексации и генерации признаков для неполных или некорректно заполненных вакансий/резюме.

Q: На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist’а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:
1) Какой датасет используют авторы?
2) Что используют в качестве признаков?
3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко

# practice

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix, coo_matrix
from implicit.nearest_neighbours import bm25_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

from additional import DataProcessor
from functools import partial
from sklearn.preprocessing import StandardScaler

## load & split

In [51]:
# load purchases
purchases = pd.read_csv('retail_train.csv')

# train/test split
test_size_weeks = 3
train = purchases[purchases['week_no'] < purchases['week_no'].max() - test_size_weeks].copy()
test = purchases[purchases['week_no'] >= purchases['week_no'].max() - test_size_weeks].copy()

# prepare true values
# true_values = test.groupby('user_id')['item_id'].unique().reset_index()
# true_values.columns=['user_id', 'actual']

EDA в отдельном блокноте

## prepare dataset

подготовим параметры обработки датасета:
* defaults: на основе кол-ва проданных товаров
* mix_feat: на комбинации стоимости и кол-ва проданных товаров

In [52]:
mix_feat_params = {
    'top_config': {'fields': ['quantity', 'sales_value'],
                   'beta': [1., 1.],
                   'k': 5000,
                   'scaler': StandardScaler},
    'uim_config': {'aggfunc': 'sum',
                #    'weights': bm25_weight
                   },
}

defaults_params = {
    'top_config': {'fields': ['quantity'],
                   'k': 5000},
    'uim_config': {'aggfunc': 'sum',
                #    'weights': bm25_weight
                   },
}

In [53]:
# создаем хранилище обучающих и валидационных данных
preparer = DataProcessor(train, test, **mix_feat_params)
preparer.fit()

## Item featuring

In [54]:
# baseline
# load items data
item_data = pd.read_csv('product.csv')
item_data.columns = item_data.columns.str.lower()
item_data.rename(columns={'product_id': 'item_id'}, inplace=True)
# dummy
item_features = pd.DataFrame(preparer.train_uim.columns)
item_features = item_features.merge(item_data, on='item_id', how='left')
item_features.set_index('item_id', inplace=True)
item_features = pd.get_dummies(item_features, columns=item_features.columns.tolist())
del item_data

## User featuring

In [55]:
# baseline user/item features
# load users data
user_data = pd.read_csv('hh_demographic.csv')
user_data.columns = user_data.columns.str.lower()
user_data.rename(columns={'household_key': 'user_id'}, inplace=True)

# dummy
user_features = pd.DataFrame(preparer.train_uim.index)
user_features = user_features.merge(user_data, on='user_id', how='left')
user_features.set_index('user_id', inplace=True)
user_features = pd.get_dummies(user_features, columns=user_features.columns.tolist())
del user_data

In [56]:
# Загружаем user features, их подготовка - в одноименном блокноте
user_data = pd.read_csv('user_features_corrected.csv')
user_features = pd.DataFrame(preparer.train_uim.index)
user_features = user_features.merge(user_data, on='user_id', how='left').fillna(0)
user_features.set_index('user_id', inplace=True)
del user_data

## LightFM

In [85]:
model = LightFM(no_components=10,
                loss='warp', # 'bpr'
                learning_rate=0.2,
                item_alpha=0.1, # смещение по товару
                user_alpha=0.1,
                random_state=42)

model.fit((preparer.train_uim_sparse > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(preparer.train_uim),
          user_features=csr_matrix(user_features.values).tocsr(),
          item_features=csr_matrix(item_features.values).tocsr(),
          epochs=15)

In [86]:
train_pr = precision_at_k(model, preparer.train_uim_sparse, k=5,
                          user_features=csr_matrix(user_features.values),
                          item_features=csr_matrix(item_features.values)).mean()

test_pr = precision_at_k(model, preparer.test_uim_sparse, k=5,
                         user_features=csr_matrix(user_features.values).tocsr(),
                         item_features=csr_matrix(item_features.values).tocsr()).mean()

print(f'Train pr@5: {train_pr}', f'Test pr@5: {test_pr}', sep='\n')


Train pr@5: 0.3733547329902649
Test pr@5: 0.003846153849735856


In [87]:
# item_index = np.arange(preparer.train_uim.columns.size)
# predictions = model.predict(user_ids=6, item_ids=item_index,
#                             user_features=csr_matrix(user_features.values).tocsr(),
#                             item_features=csr_matrix(item_features.values).tocsr(),
#                             num_threads=4)

baseline: 0.4366 / 0.0026

In [88]:
#